In [1]:
# Denpendances
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
stock = "AMD"
data = DataLoader(stock).get_data('2018-01-01', '2019-12-31')

In [3]:
def clip_dataframe(df, p):
    # Batching
    batches = len(df) // p
    max_length = batches * p
    return df.iloc[-max_length:].copy()

def simple_gain_loss(data, period):
    ndf = clip_dataframe(data, period)
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    shifted = ndf.shift(-1).fillna(method='ffill')
    target = np.array(shifted['Close']).reshape(-1, period)
    gain = target.max(1) / base - 1
    loss = target.min(1) / base - 1
    return gain, loss

In [5]:
period = 3

In [6]:
# Base sell and stop limit
gain, loss = simple_gain_loss(data, period)
sell_stop = gain.mean()
stop_loss = loss.mean()

In [7]:
# Average Daily Fluctuation
hl = data['High'] / data['Low'] - 1
minutes = ((7.5 * 60) / 15)
fluct = hl.mean() / minutes
print('Fluctation step: ', fluct)

Fluctation step:  0.0015133355901116924


In [8]:
data = DataLoader(stock).get_data('2020-01-01', '2021-12-31')

In [9]:
ndf = clip_dataframe(data, period)
# base
base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
base = np.expand_dims(base, 1)
# shifted
shift = ndf.shift(-1).fillna(method='ffill')
# High
high = np.array(shift['High']).reshape(-1, period)
# Low
low = np.array(shift['Low']).reshape(-1, period)
# Open
Open = np.array(shift['Open']).reshape(-1, period)
# Close
close = np.array(shift['Close']).reshape(-1, period)

In [10]:
# Gaining percentage
high_gains = high / base - 1
low_gains = low / base - 1
close_gains = close / base - 1
open_gains = Open / base - 1

In [11]:
# Sell and Stop limit mask
mask_high = high_gains > sell_stop
mask_low = low_gains < stop_loss
mask_open = open_gains < stop_loss

In [12]:
# Getting gained percentage 
length = len(base)
arr = []
for i in range(length):
    mo = mask_open[i]
    mh = mask_high[i]
    ml = mask_low[i]
    score = mh + 0 - ml
    if mo[0]:
        arr.append(open_gains[i, 0])
    elif score.sum() == 0:
        arr.append(close_gains[i, -1])
    else:
        for ii, s in enumerate(score):
            # Sell limit: Using this to create maximum profit
            if s == 1:
                hg = high_gains[i, ii]
                if hg > sell_stop:
                    arr.append(hg - fluct)
                else:
                    arr.append(sell_stop)
                break
            elif s == -1:
                arr.append(stop_loss)
                break

In [13]:
# Using gain to calculate possible capital gain
capital = 1000
original_capital = capital
max_shares = 100
gained = base.reshape(-1) * np.array(arr)
for i in range(length):
    g = gained[i]
    b = base.reshape(-1)[i]
    shares = capital // b
    if shares > max_shares:
        shares = max_shares
    # Update capital
    capital += g * shares

percentage_capital_gain = capital / original_capital- 1
print('Capital Gain with Current Strategy: {:.2f}%'.format(
    percentage_capital_gain * 100))


Capital Gain with Current Strategy: 665.83%
